## MNIST
<br>이제 higher level API(tf.layers 등)를 적극 활용하면서 필요에 따라 앞서 배운 low level API(tf.nn)를 활용해 세부적인 model tuning이 가능합니다. (https://goo.gl/Rmy8qq)
<br>
<br><span style="color:red;"> - 더욱 편하게 layer 를 구성할 수 있도록 돕는 **tf.layers** 를 적용합니다.
<br>- tf.layers.batch_normalization()을 활용해 손쉽게 **Batch Normalization**을 적용할 수 있습니다.</span>
<br>- BN을 적용하면 전반적으로 모델의 성능이 향상되어 Params init, Regularization, Dropout 등의 필요성이 크게 줄어듭니다. 
<br>- 물론 신경망이 깊어지고 풀어야 할 문제가 복잡해진다면 앞선 최적화 방법들을 함께 적용시켜 성능 향상을 도모할 수 있습니다. 
<br><br>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import layers

In [2]:
import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
# 각종 placeholder 들을 선언해줍니다.

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

bn_sign = tf.placeholder(tf.bool)

In [5]:
# BN 순서 : 선형 결합 -> BN 적용 -> 활성화 함수 
# activation function 을 걷어내고 BN을 먼저 적용하기 위해 dense() 대신 fully_connected()를 적용하였습니다.

L1 = tf.contrib.layers.fully_connected(X, 256, activation_fn=None)
L1 = tf.layers.batch_normalization(L1, training=bn_sign)
L1 = tf.nn.relu(L1)

L2 = tf.contrib.layers.fully_connected(L1, 256, activation_fn=None)
L2 = tf.layers.batch_normalization(L2, training=bn_sign)
L2 = tf.nn.relu(L2)

model = layers.dense(L2, 10, activation=None)

In [6]:
cost = tf.losses.softmax_cross_entropy(Y, model) 
optimizer = tf.train.AdamOptimizer(1e-3).minimize(cost) 

In [7]:
#cost = tf.losses.softmax_cross_entropy(Y, model) 

# BN 적용 중 계산되는 moving_mean & moving_variance 를 지속적으로 업데이트 해줍니다.
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(cost) 
    
# * When is_training is "True", the moving_mean and moving_variance need to be updated, 
# by default the update_ops are placed in tf.GraphKeys.UPDATE_OPS
# so they need to be added as a dependency to the train_op

In [8]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [10]:
batch_size = 100
total_batch = mnist.train.num_examples // batch_size
total_batch

550

In [11]:
# from tqdm import trange, tqdm_notebook
# for epoch in tqdm_notebook(range(15)):

for epoch in range(15):
    train_cp = [] # Training accuracy 를 동시에 출력해보도록 합니다.
    total_cost = 0 # cost

    for _ in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, 
                                                             bn_sign: True}) 
        
        # 매 Epoch마다 Total cost를 출력합니다.
        total_cost += cost_val # cost

        # 매 Epoch마다 Training accuracy를 출력합니다. (bn_sign을 False로 바꾸어 training mode가 아닌 inference mode로 실행합니다.)
        train_cp += sess.run([is_correct], feed_dict={X: batch_xs, Y: batch_ys, 
                                                      bn_sign: False}) # Training accuracy
        
    print('Epoch:', '%04d' % (epoch + 1), 
          '|| Avg. cost =', '{:.3f}'.format(total_cost / total_batch), # cost
          '|| Training accuracy : {:.3f}'.format(np.mean(train_cp))) # Training accuracy
    
print('Learning process is completed!')

Epoch: 0001 || Avg. cost = 0.197 || Training accuracy : 0.933
Epoch: 0002 || Avg. cost = 0.077 || Training accuracy : 0.982
Epoch: 0003 || Avg. cost = 0.049 || Training accuracy : 0.989
Epoch: 0004 || Avg. cost = 0.037 || Training accuracy : 0.993
Epoch: 0005 || Avg. cost = 0.028 || Training accuracy : 0.996
Epoch: 0006 || Avg. cost = 0.022 || Training accuracy : 0.997
Epoch: 0007 || Avg. cost = 0.020 || Training accuracy : 0.997
Epoch: 0008 || Avg. cost = 0.018 || Training accuracy : 0.997
Epoch: 0009 || Avg. cost = 0.015 || Training accuracy : 0.998
Epoch: 0010 || Avg. cost = 0.012 || Training accuracy : 0.998
Epoch: 0011 || Avg. cost = 0.013 || Training accuracy : 0.998
Epoch: 0012 || Avg. cost = 0.013 || Training accuracy : 0.998
Epoch: 0013 || Avg. cost = 0.009 || Training accuracy : 0.999
Epoch: 0014 || Avg. cost = 0.010 || Training accuracy : 0.999
Epoch: 0015 || Avg. cost = 0.012 || Training accuracy : 0.998
Learning process is completed!


In [12]:
# Test accuracy 를 출력합니다. 
# bn_sign을 False로 바꾸어 training mode가 아닌 [ inference mode ]로 바꿔주어야 합니다.
# 학습 단계에서는 데이터가 배치 단위로 들어오기 때문에 배치의 평균, 분산을 구하는 것이 가능하지만, 
# 테스트 단계에서는 배치 단위로 평균/분산을 구하기가 어려워 학습 단계에서 배치 단위의 평균/분산을 저장해 놓고 테스트 시에는 이를 사용합니다.
print('Test accuracy : {}'.format(sess.run(accuracy, 
                                           feed_dict={
                                               X: mnist.test.images, 
                                               Y: mnist.test.labels,
                                               bn_sign: False})))

Test accuracy : 0.9810000061988831
